In [ ]:
! pip install --upgrade pip
! pip install scanpy anndata pandas matplotlib

  Using cached pip-25.2-py3-none-any.whl.metadata (4.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.8 MB/s eta 0:00:00-:--:--
  Attempting uninstall: pip
    Found existing installation: pip 25.1
    Uninstalling pip-25.1:
      Successfully uninstalled pip-25.1
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached patsy-1.0.1-py2.py3-none-any.whl.metadata (3.3 kB)
  Using cached pynndescent-0.5.13-py3-none-any.whl.metadata (6.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.9 MB/s  0

cd astro_data \
wget -r -np -nH --cut-dirs=3 --no-check-certificate \
  https://download.brainimagelibrary.org/0c/bd/0cbd479c521afff9/

In [3]:
# ==== Astrocyte UMAP + Perturbation — normalization-only vs paper (no HVG, no scale) ====
from pathlib import Path
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from scipy import sparse

# ---------------- USER SETTINGS ----------------
COUNTS_CSV        = "astro_data/astrocytes/processed/finaltables/merfish_perturbed_cells_astrocytes.csv"  # rows=cells, cols=genes
PERT_MATRIX_CSV   = "astro_data/astrocytes/processed/finaltables/perturbation_design_astrocytes.csv"      # rows=genes, cols=cells
MIN_COUNT         = 1               # min count to call a perturbation per cell
TIE_POLICY        = "max"           # "max" or "unassigned"

POINT_SIZE        = 5               # circle size
TRIANGLE_SIZE     = 16              # triangle size for perturbed cells
CMAP_NAME         = "viridis"       # per-gene colormap
MAKE_PER_GENE_UMAPS = True          # export per-gene UMAPs
SAVE_ONLY_WITH_PERTURBED = True     # save a gene plot only if triangles exist for that gene

# UMAP (same as paper; computed on log1p of full panel)
N_PCS             = 30
N_NEIGHBORS       = 15
RANDOM_STATE      = 0
# ------------------------------------------------

# Separate outputs so we don't overwrite the paper-style results
OUT_DIR = Path("astro_out_normonly"); OUT_DIR.mkdir(exist_ok=True)
FIG_DIR = Path("figures_normonly");   FIG_DIR.mkdir(exist_ok=True)
sc.settings.figdir = str(FIG_DIR)
sc.settings.set_figure_params(dpi=120)

# 1) Load counts (rows=cells, columns=genes). Use row index as cell IDs.
df = pd.read_csv(COUNTS_CSV, header=0)
df.index = df.index.astype(str)

# split numeric (genes) vs. non-numeric (metadata)
meta_cols = [c for c in df.columns if not np.issubdtype(df[c].dtype, np.number)]
gene_cols = [c for c in df.columns if c not in meta_cols and np.issubdtype(df[c].dtype, np.number)]
if not gene_cols:
    raise ValueError("No numeric gene columns detected in COUNTS_CSV.")

# AnnData
adata = sc.AnnData(df[gene_cols].values)
adata.obs_names = df.index.astype(str)
adata.var_names = pd.Index(gene_cols, name="gene")
if meta_cols:
    adata.obs = df[meta_cols].copy()

# 2) Load perturbation matrix (genes x cells) and align to adata cells
pm = pd.read_csv(PERT_MATRIX_CSV, index_col=0)

def align_matrix_to_cells(mat: pd.DataFrame, cell_ids):
    if set(cell_ids).issubset(set(mat.columns)):
        return mat.loc[:, cell_ids]
    mt = mat.T
    if set(cell_ids).issubset(set(mt.columns)):
        return mt.loc[:, cell_ids]
    if mat.shape[1] == len(cell_ids):
        m = mat.copy(); m.columns = cell_ids; return m
    if mat.shape[0] == len(cell_ids):
        m = mat.T.copy(); m.columns = cell_ids; return m
    return None

pm2 = align_matrix_to_cells(pm, adata.obs_names.tolist())
if pm2 is None:
    raise ValueError("Could not align perturbation matrix to AnnData cells. Check IDs/order.")

# 3) One perturbation per cell (argmax + tie/threshold)
per_cell_max    = pm2.max(axis=0)
per_cell_argmax = pm2.idxmax(axis=0)
is_tie          = pm2.apply(lambda col: (col == col.max()).sum() > 1, axis=0)

labels = per_cell_argmax.astype(str).copy()
labels[per_cell_max < MIN_COUNT] = "unassigned"
if TIE_POLICY == "unassigned":
    labels[is_tie] = "unassigned"
adata.obs["perturbation"] = labels.reindex(adata.obs_names).astype(str)

# 4) **Normalization-only** preprocessing:
#    normalize_total (added) -> log1p ; no HVG, no scale (keep paper's choices)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata.copy()  # freeze log-normalized matrix for shared-scale gene plots

# 5) PCA / neighbors / UMAP on log1p(full panel)
sc.tl.pca(adata, n_comps=min(N_PCS, adata.n_vars), svd_solver="arpack", random_state=RANDOM_STATE)
sc.pp.neighbors(adata, n_neighbors=N_NEIGHBORS, n_pcs=min(N_PCS, adata.n_vars), random_state=RANDOM_STATE)
sc.tl.umap(adata, random_state=RANDOM_STATE)

# 5.5) Avoid obs/var collisions
dups = set(adata.obs.columns) & set(adata.var_names)
if dups:
    adata.obs.rename(columns={c: f"{c}__obs" for c in dups}, inplace=True)

# 6) Shared color scale for all gene plots (from adata.raw = log-normalized)
vals = adata.raw.X
if sparse.issparse(vals):
    arr = vals.data
    arr = arr[np.isfinite(arr)]
else:
    arr = np.asarray(vals).ravel()
    arr = arr[np.isfinite(arr)]
COMMON_VMIN = 0.0
COMMON_VMAX = float(np.percentile(arr, 99))
print(f"[scale] Per-gene UMAPs (norm-only) use vmin={COMMON_VMIN}, vmax={COMMON_VMAX:.3f}")

def _dense_vec(x):
    if sparse.issparse(x):
        return np.asarray(x.A).ravel()
    return np.asarray(x).ravel()

# 6a) Gray baseline — points only
adata.obs["__all"] = "cells"
fig = sc.pl.umap(
    adata, color="__all", palette=["lightgray"],
    legend_loc=None, show=False, return_fig=True, size=POINT_SIZE
)
fig.axes[0].set_title("")
fig.savefig(FIG_DIR / "umap_astro_gray_normonly.png", bbox_inches="tight", dpi=300)
plt.close(fig)
adata.obs.drop(columns="__all", inplace=True)

# 6b) Perturbation map — points only
fig = sc.pl.umap(
    adata, color="perturbation",
    legend_loc=None, show=False, return_fig=True, size=POINT_SIZE
)
fig.axes[0].set_title("")
fig.savefig(FIG_DIR / "umap_astro_perturbation_normonly.png", bbox_inches="tight", dpi=300)
plt.close(fig)

# 6c) Per-gene UMAPs — shared scale; triangles only for cells perturbed IN that gene
#     Legend outside; save only if triangles exist.
if MAKE_PER_GENE_UMAPS:
    per_gene_dir = FIG_DIR / "all_genes_common_scale_triangles_only_if_pert_normonly"
    per_gene_dir.mkdir(parents=True, exist_ok=True)

    um = adata.obsm["X_umap"]
    x, y = um[:, 0], um[:, 1]
    groups = adata.obs["perturbation"].astype(str).values
    norm = Normalize(vmin=COMMON_VMIN, vmax=COMMON_VMAX)
    cmap = plt.get_cmap(CMAP_NAME)

    for gene in adata.raw.var_names:
        expr = _dense_vec(adata.raw[:, gene].X)
        pert_mask = (groups == gene)
        n_tri = int(pert_mask.sum())
        if SAVE_ONLY_WITH_PERTURBED and n_tri == 0:
            continue

        other_mask = ~pert_mask
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.grid(False)

        # circles (others)
        sca = ax.scatter(
            x[other_mask], y[other_mask],
            s=POINT_SIZE,
            c=expr[other_mask], cmap=cmap, norm=norm,
            marker="o", edgecolors="none", alpha=0.85, zorder=1
        )
        # triangles (this gene perturbed)
        tri = ax.scatter(
            x[pert_mask], y[pert_mask],
            s=TRIANGLE_SIZE,
            c=expr[pert_mask], cmap=cmap, norm=norm,
            marker="^", edgecolors="black", linewidths=0.25,
            alpha=0.95, zorder=2, label=f"{gene} perturbed (n={n_tri})"
        )

        ax.set_xlabel("UMAP1"); ax.set_ylabel("UMAP2"); ax.set_title(gene)

        # shared colorbar
        cbar = fig.colorbar(sca, ax=ax, fraction=0.046, pad=0.04)
        cbar.ax.set_ylabel("log1p(norm counts)", rotation=270, labelpad=10)

        # legend outside (only if triangles exist)
        if n_tri > 0:
            ax.legend(handles=[tri], frameon=False, loc="upper left",
                      bbox_to_anchor=(1.2, 1.0), borderaxespad=0.0, markerscale=1.5)

        fig.savefig(per_gene_dir / f"umap_{gene}_normonly.png", bbox_inches="tight", dpi=300)
        plt.close(fig)

    print(f"[done] Saved per-gene UMAPs to {per_gene_dir}")

plt.show()

# 7) Save outputs (norm-only filenames)
adata.write(OUT_DIR / "astrocytes_final_umap_normonly.h5ad")
pd.DataFrame(adata.obsm["X_umap"], index=adata.obs_names, columns=["UMAP1","UMAP2"])\
  .to_csv(OUT_DIR / "astrocytes_umap_coords_normonly.csv")
pd.Series(adata.obs["perturbation"], name="perturbation").value_counts()\
  .to_csv(OUT_DIR / "perturbation_label_counts_normonly.csv")

print(adata)
print(f"Saved AnnData → {OUT_DIR/'astrocytes_final_umap_normonly.h5ad'}")
print(f"Saved UMAP coords → {OUT_DIR/'astrocytes_umap_coords_normonly.csv'}")
print(f"Figures in → {FIG_DIR}/")

[scale] Per-gene UMAPs (norm-only) use vmin=0.0, vmax=5.772
[done] Saved per-gene UMAPs to figures_normonly/all_genes_common_scale_triangles_only_if_pert_normonly
AnnData object with n_obs × n_vars = 14926 × 277
    obs: 'perturbation'
    uns: 'log1p', 'pca', 'neighbors', 'umap', '__all_colors', 'perturbation_colors'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'
Saved AnnData → astro_out_normonly/astrocytes_final_umap_normonly.h5ad
Saved UMAP coords → astro_out_normonly/astrocytes_umap_coords_normonly.csv
Figures in → figures_normonly/


In [ ]:
# ==== Astrocyte UMAP + Perturbation — Paper-style preprocessing (log1p only, no normalize/HVG/scale) ====
from pathlib import Path
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from scipy import sparse

# ---------------- USER SETTINGS ----------------
COUNTS_CSV        = "astro_data/astrocytes/processed/finaltables/merfish_perturbed_cells_astrocytes.csv"  # rows=cells, cols=genes
PERT_MATRIX_CSV   = "astro_data/astrocytes/processed/finaltables/perturbation_design_astrocytes.csv"      # rows=genes/guides, cols=cells

# Call per-cell perturbation
MIN_COUNT         = 1               # min count to call a perturbation per cell
TIE_POLICY        = "max"          # "max" or "unassigned"

# Paper-style QC switches (set True to enforce like paper)
APPLY_CELL_COUNT_FILTER = False     # True → filter cells by total raw counts (see thresholds below)
CELL_COUNT_MIN          = 200       # example astrocyte threshold from paper
CELL_COUNT_MAX          = 1900
APPLY_GENE_FILTER       = False     # True → keep genes with mean >= GENE_MIN_MEAN
GENE_MIN_MEAN           = 0.5       # “0.5 counts per cell” style threshold

# UMAP / plotting
POINT_SIZE        = 5               # circle size
TRIANGLE_SIZE     = 16              # triangle size for perturbed cells
CMAP_NAME         = "viridis"       # colormap for per-gene intensity
MAKE_PER_GENE_UMAPS = True          # export per-gene UMAPs
SAVE_ONLY_WITH_PERTURBED = True     # save a gene plot only if that gene has perturbed cells
RANDOM_STATE      = 0               # for PCA/UMAP reproducibility
N_PCS             = 30              # PCs to use for neighbors/UMAP (computed on log1p full matrix)
N_NEIGHBORS       = 15
# ------------------------------------------------

OUT_DIR = Path("astro_out"); OUT_DIR.mkdir(exist_ok=True)
FIG_DIR = Path("figures");   FIG_DIR.mkdir(exist_ok=True)
sc.settings.figdir = str(FIG_DIR)
sc.settings.set_figure_params(dpi=120)

# 1) Load counts (rows=cells, columns=genes). Use row index as cell IDs.
df = pd.read_csv(COUNTS_CSV, header=0)
df.index = df.index.astype(str)

# Optional paper-style QC on RAW counts (before log)
if APPLY_CELL_COUNT_FILTER:
    libsize = df.sum(axis=1)
    keep_cells = (libsize >= CELL_COUNT_MIN) & (libsize <= CELL_COUNT_MAX)
    df = df.loc[keep_cells]
if APPLY_GENE_FILTER:
    mean_counts = df.mean(axis=0)
    keep_genes = mean_counts >= GENE_MIN_MEAN
    df = df.loc[:, keep_genes]

# split numeric (genes) vs. non-numeric (metadata)
meta_cols = [c for c in df.columns if not np.issubdtype(df[c].dtype, np.number)]
gene_cols = [c for c in df.columns if c not in meta_cols and np.issubdtype(df[c].dtype, np.number)]
if not gene_cols:
    raise ValueError("No numeric gene columns detected in COUNTS_CSV after filtering.")

# AnnData
adata = sc.AnnData(df[gene_cols].values)
adata.obs_names = df.index.astype(str)
adata.var_names = pd.Index(gene_cols, name="gene")
if meta_cols:
    adata.obs = df[meta_cols].copy()

# 2) Load perturbation matrix (genes x cells) and align to adata cells
pm = pd.read_csv(PERT_MATRIX_CSV, index_col=0)

def align_matrix_to_cells(mat: pd.DataFrame, cell_ids):
    if set(cell_ids).issubset(set(mat.columns)):
        return mat.loc[:, cell_ids]
    mt = mat.T
    if set(cell_ids).issubset(set(mt.columns)):
        return mt.loc[:, cell_ids]
    if mat.shape[1] == len(cell_ids):
        m = mat.copy(); m.columns = cell_ids; return m
    if mat.shape[0] == len(cell_ids):
        m = mat.T.copy(); m.columns = cell_ids; return m
    return None

pm2 = align_matrix_to_cells(pm, adata.obs_names.tolist())
if pm2 is None:
    raise ValueError("Could not align perturbation matrix to AnnData cells. Check IDs/order.")

# 3) One perturbation per cell (argmax + tie/threshold)
per_cell_max    = pm2.max(axis=0)
per_cell_argmax = pm2.idxmax(axis=0)
is_tie          = pm2.apply(lambda col: (col == col.max()).sum() > 1, axis=0)

labels = per_cell_argmax.astype(str).copy()
labels[per_cell_max < MIN_COUNT] = "unassigned"
if TIE_POLICY == "unassigned":
    labels[is_tie] = "unassigned"

adata.obs["perturbation"] = labels.reindex(adata.obs_names).astype(str)

# 4) PAPER-STYLE PREPROCESSING → log1p ONLY (no normalize_total, no HVG, no scale)
# Keep a copy of log1p matrix in .raw for plotting with a shared color scale
sc.pp.log1p(adata)
adata.raw = adata.copy()  # raw = log1p(full gene set), used for per-gene plotting and scale

# 5) PCA / neighbors / UMAP on the log1p matrix (entire gene panel)
#    (no HVG subsetting, no scaling)
sc.tl.pca(adata, n_comps=min(N_PCS, adata.n_vars), svd_solver="arpack", random_state=RANDOM_STATE)
sc.pp.neighbors(adata, n_neighbors=N_NEIGHBORS, n_pcs=min(N_PCS, adata.n_vars), random_state=RANDOM_STATE)
sc.tl.umap(adata, random_state=RANDOM_STATE)

# 5.5) Avoid obs/var name collisions
dups = set(adata.obs.columns) & set(adata.var_names)
if dups:
    adata.obs.rename(columns={c: f"{c}__obs" for c in dups}, inplace=True)

# 6) Shared color scale for all gene plots (from adata.raw = log1p matrix)
vals = adata.raw.X
if sparse.issparse(vals):
    arr = vals.data
    arr = arr[np.isfinite(arr)]
else:
    arr = np.asarray(vals).ravel()
    arr = arr[np.isfinite(arr)]
COMMON_VMIN = 0.0
COMMON_VMAX = float(np.percentile(arr, 99))  # robust to outliers
print(f"[scale] Per-gene UMAPs use vmin={COMMON_VMIN}, vmax={COMMON_VMAX:.3f}")

def _dense_vec(x):
    if sparse.issparse(x):
        return np.asarray(x.A).ravel()
    return np.asarray(x).ravel()

# 7) Plots (same visuals you requested)
# 7a) Gray baseline — points only, no legend/text
adata.obs["__all"] = "cells"
fig = sc.pl.umap(
    adata, color="__all", palette=["lightgray"],
    legend_loc=None, frameon=False, show=False, return_fig=True, size=POINT_SIZE
)
fig.axes[0].set_title("")
fig.savefig(FIG_DIR / "umap_astro_gray.png", bbox_inches="tight", dpi=300)
plt.close(fig)
adata.obs.drop(columns="__all", inplace=True)

# 7b) Perturbation overview — points only (no legend/text)
fig = sc.pl.umap(
    adata, color="perturbation",
    legend_loc=None, show=False, return_fig=True, size=POINT_SIZE
)
fig.axes[0].set_title("")
fig.savefig(FIG_DIR / "umap_astro_perturbation.png", bbox_inches="tight", dpi=300)
plt.close(fig)

# 7c) Per-gene UMAPs — shared color scale; triangles only for cells perturbed IN that gene
if MAKE_PER_GENE_UMAPS:
    per_gene_dir = FIG_DIR / "all_genes_common_scale_triangles_paper_style"
    per_gene_dir.mkdir(parents=True, exist_ok=True)

    um = adata.obsm["X_umap"]
    x, y = um[:, 0], um[:, 1]
    groups = adata.obs["perturbation"].astype(str).values
    norm = Normalize(vmin=COMMON_VMIN, vmax=COMMON_VMAX)
    cmap = plt.get_cmap(CMAP_NAME)

    for gene in adata.raw.var_names:
        expr = _dense_vec(adata.raw[:, gene].X)
        pert_mask = (groups == gene)
        n_tri = int(pert_mask.sum())
        if SAVE_ONLY_WITH_PERTURBED and n_tri == 0:
            continue  # skip genes with no matching perturbation cells

        other_mask = ~pert_mask

        fig, ax = plt.subplots(figsize=(6, 6))
        ax.grid(False)

        # circles (others)
        sca = ax.scatter(
            x[other_mask], y[other_mask],
            s=POINT_SIZE,
            c=expr[other_mask], cmap=cmap, norm=norm,
            marker="o", edgecolors="none", alpha=0.85, zorder=1
        )
        # triangles (this gene perturbed)
        tri = ax.scatter(
            x[pert_mask], y[pert_mask],
            s=TRIANGLE_SIZE,
            c=expr[pert_mask], cmap=cmap, norm=norm,
            marker="^", edgecolors="black", linewidths=0.25,
            alpha=0.95, zorder=2, label=f"{gene} perturbed (n={n_tri})"
        )

        ax.set_xlabel("UMAP1"); ax.set_ylabel("UMAP2"); ax.set_title(gene)

        # shared colorbar
        cbar = fig.colorbar(sca, ax=ax, fraction=0.046, pad=0.04)
        cbar.ax.set_ylabel("log1p(counts)", rotation=270, labelpad=10)

        # legend outside (only if triangles exist)
        if n_tri > 0:
            ax.legend(handles=[tri], frameon=False, loc="upper left",
                      bbox_to_anchor=(1.2, 1.0), borderaxespad=0.0, markerscale=1.5)

        fig.savefig(per_gene_dir / f"umap_{gene}.png", bbox_inches="tight", dpi=300)
        plt.close(fig)

    print(f"[done] Saved per-gene UMAPs to {per_gene_dir}")

plt.show()

# 8) Save outputs
adata.write(OUT_DIR / "astrocytes_final_umap.h5ad")
pd.DataFrame(adata.obsm["X_umap"], index=adata.obs_names, columns=["UMAP1","UMAP2"])\
  .to_csv(OUT_DIR / "astrocytes_umap_coords.csv")
pd.Series(adata.obs["perturbation"], name="perturbation").value_counts()\
  .to_csv(OUT_DIR / "perturbation_label_counts.csv")

print(adata)
print(f"Saved AnnData → {OUT_DIR/'astrocytes_final_umap.h5ad'}")
print(f"Saved UMAP coords → {OUT_DIR/'astrocytes_umap_coords.csv'}")
print(f"Figures in → {FIG_DIR}/")


[scale] Per-gene UMAPs use vmin=0.0, vmax=3.434
[done] Saved per-gene UMAPs to figures/all_genes_common_scale_triangles_paper_style
AnnData object with n_obs × n_vars = 14926 × 277
    obs: 'perturbation'
    uns: 'log1p', 'pca', 'neighbors', 'umap', '__all_colors', 'perturbation_colors'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'
Saved AnnData → astro_out/astrocytes_final_umap.h5ad
Saved UMAP coords → astro_out/astrocytes_umap_coords.csv
Figures in → figures/
